In [3]:
from keras.datasets import mnist
import matplotlib.pyplot as  plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_blobs
from sklearn import mixture
from sklearn.preprocessing import OrdinalEncoder

ModuleNotFoundError: No module named 'keras'

In [3]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(28, 28),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 56))

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(y_train[:10])

print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

plot_digits(x_train)

In [ ]:

pca = PCA().fit(x_train.reshape(60000,28*28))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')


In [ ]:
pca = PCA(0.5)  # project from 64 to 2 dimensions

np.random.seed(42)
noise = np.random.normal(size=x_train.shape,scale=50)
noisy=x_train+noise
plot_digits(noisy)

components = pca.fit_transform(noisy.reshape(60000,28*28))
filtered = pca.inverse_transform(components)
plot_digits(filtered)


In [ ]:
pca = PCA()  # project from 64 to 2 dimensions

components = pca.fit_transform(noisy.reshape(60000,28*28))
filtered = pca.inverse_transform(components)

noise=pca.explained_variance_ratio_
plt.plot(np.cumsum(noise))
plt.xlabel('number of components')
plt.ylabel('level of impurity or noise');

In [ ]:
def add_salt_pepper_noise(X_imgs,salt_vs_pepper=0.9,amount=0.1):
    # Need to produce a copy as to not modify the original image
    X_imgs_copy = X_imgs.copy()
    row, col = X_imgs_copy[0].shape
    num_salt = np.ceil(amount * X_imgs_copy[0].size * salt_vs_pepper)
    num_pepper = np.ceil(amount * X_imgs_copy[0].size * (1.0 - salt_vs_pepper))
    
    for X_img in X_imgs_copy:
        # Add Salt noise
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in X_img.shape]
        X_img[coords[0], coords[1]] = 255

        # Add Pepper noise
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in X_img.shape]
        X_img[coords[0], coords[1]] = 0
    return X_imgs_copy

In [ ]:

salt_pepper_noise_imgs = add_salt_pepper_noise(x_train)
plot_digits(salt_pepper_noise_imgs)

In [ ]:
pca = PCA(0.43)  # project from 64 to 2 dimensions


components = pca.fit_transform(salt_pepper_noise_imgs.reshape(60000,28*28))
filtered = pca.inverse_transform(components)
plot_digits(filtered)


In [ ]:
pca = PCA()  # project from 64 to 2 dimensions


components = pca.fit_transform(salt_pepper_noise_imgs.reshape(60000,28*28))
filtered = pca.inverse_transform(components)
noise=pca.explained_variance_ratio_
plt.plot(np.cumsum(noise))
plt.xlabel('number of components')
plt.ylabel('level of impurity or noise')
plt.show()

In [ ]:
def Generate_data_with_labels_1(n=100,range=1000,noise=300):
  x1 = np.random.randint(range,size=n).reshape(n,1)
  
  noise2=np.random.normal(5,noise,size=n).reshape(n,1)
  noise3=np.random.normal(50,noise,size=n).reshape(n,1)
  noise4=np.random.normal(100,noise,size=n).reshape(n,1)
  
  x2=x1*3 +noise2
  x3=-x1*15 +noise3
  x4=x1*1 +noise4
  
  x=np.concatenate((x1,x2,x3,x4),axis=1)
  y=np.array(x2<=range/2).astype(int).reshape(-1,)
  
  X=pd.DataFrame(x)
  Y=pd.DataFrame(y)
  
  x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state=42)

  return x_train, x_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = Generate_data_with_labels_1()



sns.pairplot(x_train)


In [ ]:
def test_lda_apply_decorrelate_test_lda(x_train, y_train,x_test, y_test):
  clf = LinearDiscriminantAnalysis()
  clf.fit(x_train, y_train)
  print('score : ',clf.score(x_test, y_test))
  
  pca = PCA()  
  components = pca.fit_transform(x_train)
  sns.scatterplot(components[:,0],components[:,1],hue=y_train.to_numpy().reshape(-1,))
  plt.show()

  clf = LinearDiscriminantAnalysis('svd')
  clf.fit_transform(components, y_train)
  print(clf.score(components, y_train))
  


In [ ]:
test_lda_apply_decorrelate_test_lda(x_train, y_train,x_test, y_test)

In [ ]:
def Generate_data_with_labels_2(num_samples = 400):
  # The desired mean values of the sample.
  mu = np.array([5.0, 0.0, 10.0])

  # The desired covariance matrix.
  r = np.array([
        [  3.40, -2.75, -2.00],
        [ -2.75,  5.50,  1.50],
        [ -2.00,  1.50,  1.25]
    ])
  
  
  # Generate the random samples.
  np.random.seed(42)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  # Generate the labels.
  #line=(x[:,1]*0.3)+10
  split=(x[:,2]<=9)+(x[:,2]>=11)
  #split=(x[:,2]<=9)*1+(x[:,2]>=11)*2+((x[:,2]<11)&(x[:,2]>9))*3
  y=np.array(split).astype(int).reshape(-1,)


  X=pd.DataFrame(x)
  Y=pd.DataFrame(y)
  
  x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state=42)

 return x_train, x_test, y_train, y_test
  


In [ ]:
x_train, x_test, y_train, y_test = Generate_data_with_labels_2()
sns.pairplot(x_train)

In [ ]:
test_lda_apply_decorrelate_test_lda(x_train, y_train,x_test, y_test)

In [ ]:
def Generate_data_with_labels_3(n_samples = 1500,random_state = 170):
  x, y = make_blobs(n_samples=n_samples, random_state=random_state)
  # Anisotropicly distributed data
  transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
  X_aniso = np.dot(x, transformation)
  gmm =mixture.GaussianMixture(n_components=3, covariance_type='full')
  gmm.fit(X_aniso)
  y_pred =gmm.predict(X_aniso)
  
  X=pd.DataFrame(X_aniso)
  Y=pd.DataFrame(y_pred)
  
  x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state=42)

  return x_train, x_test, y_train, y_test
  


In [ ]:
x_train, x_test, y_train, y_test = Generate_data_with_labels_3()
sns.scatterplot(x_train[0].to_numpy().reshape(-1,),x_train[1].to_numpy().reshape(-1,),hue=y_train.to_numpy().reshape(-1,))

In [ ]:
test_lda_apply_decorrelate_test_lda(x_train, y_train,x_test, y_test)

In [ ]:
def Generate_data_with_labels_4():
  penguins = sns.load_dataset("penguins")
  penguins = penguins.replace([np.inf, -np.inf], np.nan)
  penguins = penguins.fillna(penguins.mean())
  x=penguins.drop(columns=['species','island','sex'])
  y=((penguins['species']=='Adelie')*1)+((penguins['species']=='Chinstrap')*2)+((penguins['species']=='Gentoo')*3)

  X=pd.DataFrame(x)
  Y=pd.DataFrame(y)
  
  x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state=42)

  return x_train, x_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = Generate_data_with_labels_4()


penguins = sns.load_dataset("penguins")
sns.pairplot(penguins,hue='species')


In [ ]:
test_lda_apply_decorrelate_test_lda(x_train, y_train,x_test, y_test)